In [64]:
import os.path

import parameter
import query
import time
import copy
from model import Model
from station import Station
import numpy as np
import pandas as pd
from datetime import datetime , timedelta
import matplotlib.pyplot as plt

In [65]:
# parameter.get_station_information(True)

In [66]:
stations = pd.read_csv('data/stations.csv', index_col=0)
tph = 4
path = f'data/station_data/test_whitelist/v1/tph_{tph}/'
if not os.path.exists(path):
    os.makedirs(path)    
station_information = pd.read_csv('data/station_data/status_at_time/2024_6_17_13:8.csv', index_col=0)

whitelist = ['Allen St & Rivington St', 'Allen St & Stanton St', '1 Ave & E 16 St', 'Mercer St & Spring St', 'Mercer St & Bleecker St', '9 Ave & W 18 St', 'E 103 St & Lexington Ave', '1 Ave & E 6 St', 'Norfolk St & Broome St', 'E 1 St & 1 Ave', 'S 4 St & Wythe Ave', 'Lafayette St & Jersey St', 'Broadway & Berry St', 'S 5 Pl & S 5 St', 'E 5 St & Cooper Sq', 'Howard St & Lafayette St', 'E 32 St & Park Ave', 'Greenwich St & Hubert St', 'W 27 St & 7 Ave', 'Vesey Pl & River Terrace']

now = time.time()
station_list = {}
start_date = query.get_datetime(2023, 5, 1, 0, 0, 0)
end_date = query.get_datetime(2023, 6, 30, 0, 0, 0)
# for station in whitelist:
#     station_list[station] = parameter.get_station(station, station_information = station_information,
#                                                   start_date=start_date, end_date=end_date, whitelist=whitelist, tph=1)
#     print(station, 'done')
#     parameter.pickle_station(station_list[station], path)

In [67]:
station_list1 = {}
path1 = f'data/station_data/test_whitelist/v1/'
tph = 12
for station in whitelist:
    station_list1[station] = parameter.get_pickle_station(station, path1)
    station_list1[station].refine_by_3()
station_list0 = copy.deepcopy(station_list1)
station_list2 = copy.deepcopy(station_list1)
model_0 = Model(station_names=whitelist, stations_dict=station_list0, in_transit=[], tph=tph)
model_0.init_state(path='data/station_data/status_at_time/2024_6_17_14:6.csv', time=timedelta(hours=14))
model_1 = Model(station_names=whitelist, stations_dict=station_list1, in_transit=[], tph=tph)
model_1.init_state(path='data/station_data/status_at_time/2024_6_17_14:6.csv', time=timedelta(hours=14))
model_2 = Model(station_names=whitelist, stations_dict=station_list2, in_transit=[], tph=tph)
model_2.init_state(path='data/station_data/status_at_time/2024_6_17_15:5.csv', time=timedelta(hours=15))


In [118]:
model_1.sim()
station_bikes = 0
for station in model_1.stations_dict.values():
    print(station.name, 'has', station.curr_bikes, 'bikes')
    station_bikes += station.curr_bikes
print(len(model_1.in_transit), 'bikes in transit')
print(station_bikes, 'station bikes')
print(len(model_1.in_transit)+station_bikes, 'total bikes')
print('time:', model_1.curr_time, 'total_trips:', model_1.total_trips, 'failures:', model_1.failures)
print(model_1.mean_sq_error(model_2.stations_dict))
print(model_0.mean_sq_error(model_2.stations_dict))

Allen St & Rivington St has 38 bikes
Allen St & Stanton St has 28 bikes
1 Ave & E 16 St has 8 bikes
Mercer St & Spring St has 0 bikes
Mercer St & Bleecker St has 32 bikes
9 Ave & W 18 St has 8 bikes
E 103 St & Lexington Ave has 20 bikes
1 Ave & E 6 St has 83 bikes
Norfolk St & Broome St has 0 bikes
E 1 St & 1 Ave has 16 bikes
S 4 St & Wythe Ave has 0 bikes
Lafayette St & Jersey St has 11 bikes
Broadway & Berry St has 0 bikes
S 5 Pl & S 5 St has 36 bikes
E 5 St & Cooper Sq has 12 bikes
Howard St & Lafayette St has 0 bikes
E 32 St & Park Ave has 5 bikes
Greenwich St & Hubert St has 0 bikes
W 27 St & 7 Ave has 6 bikes
Vesey Pl & River Terrace has 0 bikes
257 bikes in transit
303 station bikes
560 total bikes
time: 18:00:00 total_trips: 3333 failures: 993
1004.0
9.95


In [120]:
for station in whitelist:
    print('init', model_0.stations_dict[station].curr_bikes, 'pred', model_1.stations_dict[station].curr_bikes, 'res', model_2.stations_dict[station].curr_bikes, model_0.stations_dict[station].rate[model_1.curr_tick])
print(model_1.mean_sq_error(model_2.stations_dict))
print(model_0.mean_sq_error(model_2.stations_dict))


init 14 pred 38 res 11 4.4222222222222225
init 9 pred 28 res 12 4.511111111111111
init 1 pred 8 res 0 4.177777777777778
init 61 pred 0 res 53 5.5777777777777775
init 65 pred 32 res 59 5.066666666666666
init 75 pred 8 res 76 6.666666666666667
init 20 pred 20 res 21 1.0888888888888888
init 14 pred 83 res 20 3.3555555555555556
init 32 pred 0 res 33 6.088888888888889
init 34 pred 16 res 33 4.955555555555556
init 5 pred 0 res 7 3.8444444444444446
init 57 pred 11 res 55 7.022222222222222
init 5 pred 0 res 6 2.8444444444444446
init 35 pred 36 res 34 2.1777777777777776
init 35 pred 12 res 33 2.3777777777777778
init 34 pred 0 res 33 3.9555555555555557
init 8 pred 5 res 4 5.0
init 5 pred 0 res 5 17.533333333333335
init 4 pred 6 res 4 7.066666666666666
init 47 pred 0 res 44 10.844444444444445
1004.0
9.95


In [20]:
# now = time.time()
# stations = pd.read_csv('data/stations.csv', index_col=0)
# station_list = {}
# for station in stations['station_name']:
#     station_list[station] = parameter.get_pickle_station(station)
# remove = {}
# for station in station_list.values():
#     for i in range(len(station.transition)):
#         if sum(station.transition[i]) == 0:
#             station.transition[i][0] = 1
# print(time.time() - now)